<a href="https://colab.research.google.com/github/JuanJoseMoraCarrascal/algoritmos/blob/main/Algoritmos_de_busqueda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import heapq

# Clase para representar un nodo en el árbol de búsqueda
class Node:
    def __init__(self, state, parent=None, action=None):
        self.state = state  # Estado del nodo
        self.parent = parent  # Nodo padre
        self.action = action  # Acción tomada para llegar a este nodo

    # Método para reconstruir el camino de acciones desde el nodo inicial hasta el nodo actual
    def actions_path(self):
        actions = []  # Lista para almacenar las acciones
        node = self
        while node.parent is not None:  # Retrocedemos desde el nodo actual hasta el nodo raíz
            actions.append(node.action)
            node = node.parent
        actions.reverse()  # Invertimos el orden para que sea desde el inicio
        return actions

    # Método para reconstruir el camino de estados desde el nodo inicial hasta el nodo actual
    def states_path(self):
        states = []  # Lista para almacenar los estados
        node = self
        while node.parent is not None:  # Retrocedemos desde el nodo actual hasta el nodo raíz
            states.append(node.state)
            node = node.parent
        states.reverse()  # Invertimos el orden para que sea desde el inicio
        return states

# Clase para representar un laberinto
class Maze:
    def __init__(self, matrix):
        self.matrix = np.array(matrix)  # Convertimos la matriz en un arreglo de numpy
        self.num_rows, self.num_cols = self.matrix.shape  # Dimensiones del laberinto

    # Método para verificar si un movimiento es válido
    def is_valid_move(self, row, col):
        return (0 <= row < self.num_rows and  # Dentro de los límites del laberinto
                0 <= col < self.num_cols and
                self.matrix[row, col] == 0)  # Espacio libre

# Clase que define el problema de búsqueda
class SearchProblem:
    def __init__(self, initial, goal, maze):
        self.initial = initial  # Estado inicial
        self.goal = goal  # Estado objetivo
        self.maze = maze  # Instancia del laberinto

    # Genera las acciones posibles desde un nodo
    def actions(self, node):
        actions = []
        x, y = node.state
        # Verificamos las direcciones posibles: derecha, izquierda, abajo, arriba
        if y < self.maze.num_cols - 1 and self.maze.is_valid_move(x, y + 1):
            actions.append('Derecha')
        if y > 0 and self.maze.is_valid_move(x, y - 1):
            actions.append('Izquierda')
        if x < self.maze.num_rows - 1 and self.maze.is_valid_move(x + 1, y):
            actions.append('Abajo')
        if x > 0 and self.maze.is_valid_move(x - 1, y):
            actions.append('Arriba')
        return actions

    # Devuelve el nodo resultante de aplicar una acción a un nodo dado
    def result(self, node, action):
        x, y = node.state
        if action == 'Arriba':
            new_state = (x - 1, y)
        elif action == 'Abajo':
            new_state = (x + 1, y)
        elif action == 'Izquierda':
            new_state = (x, y - 1)
        elif action == 'Derecha':
            new_state = (x, y + 1)
        return Node(state=new_state, parent=node, action=action)

    # Verifica si un nodo es el objetivo
    def is_goal(self, node):
        return node.state == self.goal

# Expande un nodo generando sus hijos
def expand(node, problem):
    children = []
    for action in problem.actions(node):  # Para cada acción posible
        child = problem.result(node, action)  # Genera un nodo hijo
        children.append(child)
    return children


# Implementación de Búsqueda por Amplitud (BFS)
def BFS(problem):
    closed = set()  # Conjunto para almacenar nodos ya explorados
    fringe = [Node(state=problem.initial)]  # Cola inicial con el nodo inicial
    explored_nodes = []  # Lista para registrar los nodos explorados

    while fringe:
        node = fringe.pop(0)  # Extraemos el primer nodo de la cola
        explored_nodes.append(node.state)

        if problem.is_goal(node):  # Si es el nodo objetivo
            return node.actions_path(), node.states_path(), explored_nodes

        if node.state not in closed:  # Si el nodo no ha sido explorado
            closed.add(node.state)
            for child in expand(node, problem):  # Generamos y añadimos los hijos a la cola
                fringe.append(child)

    return None, None, explored_nodes  # Si no hay solución, retornamos valores vacíos


# Implementación de Búsqueda por Profundidad (DFS)
def DFS(problem):
    # Pila de nodos a explorar (LIFO)
    stack = [Node(problem.initial)]
    explored = set()  # Conjunto para almacenar estados de nodos explorados
    explored_nodes = []  # Lista para registrar el orden de los nodos explorados

    while stack:
        # Sacamos el nodo actual del tope de la pila
        node = stack.pop()

        # Si el nodo ya ha sido explorado, pasamos al siguiente
        if node.state in explored:
            continue

        # Marcar el nodo como explorado
        explored.add(node.state)
        # Añadir el nodo actual a la lista de nodos explorados
        explored_nodes.append(node.state)

        # Si el nodo es el objetivo, devolver el camino de acciones, estados y nodos explorados
        if problem.is_goal(node):
            return node.actions_path(), node.states_path(), explored_nodes

        # Expandir los nodos hijos y añadirlos a la pila si no han sido explorados
        for child in expand(node, problem):
            if child.state not in explored and child not in stack:
                stack.append(child)  # Añadir el hijo a la pila

    return None, None, explored_nodes  # Si no hay solución, devolver None para caminos y los nodos explorados




# Implementación de Búsqueda por Costo Uniforme (UCS)
def ucs(maze: Maze, start: tuple, goal: tuple):
    queue = [(0, start)]  # Cola de prioridad inicial con el costo y la posición inicial
    costs = {start: 0}  # Diccionario para almacenar el costo acumulado hacia cada nodo
    parents = {start: None}  # Diccionario para rastrear los nodos padres
    moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Movimientos posibles: arriba, abajo, izquierda, derecha
    explored_nodes = []  # Lista para registrar los nodos explorados

    while queue:
        current_cost, (row, col) = heapq.heappop(queue)  # Extraemos el nodo con menor costo
        explored_nodes.append((row, col))  # Registramos el nodo explorado

        if (row, col) == goal:  # Si alcanzamos el nodo objetivo
            path = []
            while (row, col) is not None:  # Reconstruimos el camino desde el objetivo hacia el inicio
                path.append((row, col))
                if parents[(row, col)] is None:  # Verificamos si alcanzamos el nodo inicial
                    break
                row, col = parents[(row, col)]
            return path[::-1], current_cost, explored_nodes  # Devolvemos el camino, costo total y nodos explorados

        for dr, dc in moves:  # Generamos las posiciones vecinas
            new_row, new_col = row + dr, col + dc
            new_position = (new_row, new_col)

            if maze.is_valid_move(new_row, new_col):  # Verificamos si la posición es válida
                new_cost = current_cost + 1  # Actualizamos el costo acumulado

                # Si es un nodo nuevo o encontramos un costo menor
                if new_position not in costs or new_cost < costs[new_position]:
                    costs[new_position] = new_cost  # Actualizamos el costo
                    parents[new_position] = (row, col)  # Actualizamos el padre del nodo
                    heapq.heappush(queue, (new_cost, new_position))  # Añadimos el nodo a la cola de prioridad

    return [], float('inf'), explored_nodes  # Si no hay solución, devolvemos valores vacíos

# Heurística para búsqueda informada (distancia Manhattan)
def heuristic(a: tuple, b: tuple):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])  # Distancia Manhattan entre dos puntos


# Implementación de Búsqueda Greedy
def greedy_search(maze: Maze, start: tuple, goal: tuple):
    priority_queue = []  # Cola de prioridad basada en la heurística
    heapq.heappush(priority_queue, (heuristic(start, goal), start))  # Insertamos el nodo inicial
    visited = set()  # Conjunto para rastrear los nodos visitados
    parents = {start: None}  # Diccionario para rastrear los nodos padres
    explored_nodes = []  # Lista para registrar los nodos explorados

    while priority_queue:
        _, current = heapq.heappop(priority_queue)  # Extraemos el nodo con menor heurística

        if current == goal:  # Si alcanzamos el nodo objetivo
            path = []
            while current is not None:  # Reconstruimos el camino desde el objetivo hacia el inicio
                path.append(current)
                current = parents[current]
            return path[::-1], explored_nodes  # Devolvemos el camino y los nodos explorados

        visited.add(current)  # Marcamos el nodo como visitado
        explored_nodes.append(current)

        row, col = current
        neighbors = [(row - 1, col), (row + 1, col), (row, col - 1), (row, col + 1)]  # Generamos las posiciones vecinas

        for neighbor in neighbors:
            r, c = neighbor

            if maze.is_valid_move(r, c) and neighbor not in visited:  # Verificamos si es válido y no visitado
                parents[neighbor] = current  # Actualizamos el padre del nodo
                heapq.heappush(priority_queue, (heuristic(neighbor, goal), neighbor))  # Añadimos a la cola de prioridad

    return [], explored_nodes  # Si no hay solución, devolvemos valores vacíos


# Implementación de Búsqueda A* (A-Star)
def a_star(maze: Maze, start: tuple, goal: tuple):
    open_list = []  # Cola de prioridad para los nodos abiertos
    heapq.heappush(open_list, (0 + heuristic(start, goal), start))  # Insertamos el nodo inicial con su costo f
    came_from = {}  # Diccionario para rastrear el nodo desde el cual llegamos a cada estado
    g_score = {start: 0}  # Diccionario para almacenar el costo g (distancia desde el inicio)
    f_score = {start: heuristic(start, goal)}  # Diccionario para almacenar el costo f (g + h)
    closed_set = set()  # Conjunto de nodos explorados
    explored_nodes = []  # Lista para registrar los nodos explorados

    while open_list:
        current_f_score, current = heapq.heappop(open_list)  # Extraemos el nodo con menor costo f
        explored_nodes.append(current)  # Registramos el nodo explorado

        if current == goal:  # Si alcanzamos el nodo objetivo
            path = []
            while current in came_from:  # Reconstruimos el camino desde el objetivo hacia el inicio
                path.append(current)
                current = came_from[current]
            return path[::-1], explored_nodes  # Devolvemos el camino y los nodos explorados

        closed_set.add(current)  # Marcamos el nodo como explorado
        row, col = current
        neighbors = [(row - 1, col), (row + 1, col), (row, col - 1), (row, col + 1)]  # Generamos las posiciones vecinas

        for neighbor in neighbors:
            r, c = neighbor

            if maze.is_valid_move(r, c):  # Verificamos si la posición es válida
                if neighbor in closed_set:  # Si el vecino ya fue explorado, lo ignoramos
                    continue

                tentative_g_score = g_score[current] + 1  # Calculamos el costo g provisional

                # Si es un nodo nuevo o encontramos un costo g menor
                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current  # Actualizamos el nodo padre
                    g_score[neighbor] = tentative_g_score  # Actualizamos el costo g
                    f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)  # Actualizamos el costo f

                    if neighbor not in [i[1] for i in open_list]:  # Si el vecino no está en la lista abierta
                        heapq.heappush(open_list, (f_score[neighbor], neighbor))  # Añadimos a la lista abierta
                    print("Busqueda A* Exploracion Distacia Manhattan:")
                    # Imprimir la información del vecino como lo solicitaste
                    print(f"Exploración del vecino frontera {neighbor}, acción: ", end="")

                    if r == row - 1:
                        print("Arriba")
                    elif r == row + 1:
                        print("Abajo")
                    elif c == col - 1:
                        print("Izquierda")
                    elif c == col + 1:
                        print("Derecha")

                    print(f"  g = (distancia desde inicio): {tentative_g_score}")
                    print(f"  h = (heurística, distancia estimada a ¡Goal!): {heuristic(neighbor, goal)}")
                    print(f"  f = (costo total, g + h): {f_score[neighbor]}")
                    print("----------------------------------------")

    return None, explored_nodes  # Si no hay solución, devolvemos valores vacíos

# Función para obtener las acciones desde una lista de posiciones del camino
def obtener_camino_de_acciones(path):
    acciones = []  # Lista para almacenar las acciones necesarias
    # Diccionario que mapea las diferencias entre posiciones con las acciones correspondientes
    direcciones = {
        (-1, 0): "Arriba",   # Movimiento hacia arriba
        (1, 0): "Abajo",     # Movimiento hacia abajo
        (0, -1): "Izquierda",# Movimiento hacia la izquierda
        (0, 1): "Derecha"    # Movimiento hacia la derecha
    }

    for i in range(1, len(path)):  # Iteramos desde el segundo nodo hasta el último
        delta_row = path[i][0] - path[i-1][0]  # Calculamos la diferencia en filas
        delta_col = path[i][1] - path[i-1][1]  # Calculamos la diferencia en columnas
        accion = direcciones.get((delta_row, delta_col))  # Obtenemos la acción correspondiente
        if accion:  # Si existe una acción válida
            acciones.append(accion)  # Añadimos la acción a la lista

    return acciones  # Retornamos la lista de acciones

# Definimos el laberinto donde 0 es un espacio libre y 1 es una pared.
maze_matrix = [
    [0, 1, 0, 0, 0, 0],
    [0, 1, 0, 1, 1, 0],
    [0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 1],
    [0, 1, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0]
]


# Crear una instancia del laberinto
maze = Maze(maze_matrix)

# Definir las coordenadas de inicio y objetivo
start_position = (0, 0)  # Posición inicial en la esquina superior izquierda
goal_position = (7, 5)   # Posición objetivo en la esquina inferior derecha

# Crear el problema de búsqueda con el laberinto, el inicio y el objetivo
problem = SearchProblem(start_position, goal_position, maze)

# Ejecutar BFS
# Ejecutar DFS y obtener el camino hacia el objetivo y los nodos explorados


# Ejecutar BFS
actions_path_bfs, state_path_bfs, nodes_explored_bfs = BFS(problem)
print("\n===== Búsqueda por Amplitud (BFS) =====")
if actions_path_bfs is not None and state_path_bfs is not None:
    print("Movimientos:", actions_path_bfs)
    print("Estados del camino:", state_path_bfs)
else:
    print("No se encontró solución.")
print("Nodos explorados:", nodes_explored_bfs)


def dfs_search(problem):
    actions_path, states_path, explored_nodes = DFS(problem)

    print("\n===== Búsqueda por Profundidad (DFS) =====")
    if actions_path is not None and states_path is not None:
        print("Movimientos:", actions_path)
        print("Camino de estados:", states_path)
        print("Nodos explorados:", explored_nodes)
    else:
        print("No se encontró solución.")
dfs_search(problem)

# Ejecutar UCS
path_ucs, cost_ucs, nodes_explored_ucs = ucs(maze, start_position, goal_position)
print("\n===== Búsqueda por Costo Uniforme (UCS) =====")
if path_ucs:
    print("Camino más corto:", path_ucs)
    print("Costo total:", cost_ucs)
    camino_de_acciones = obtener_camino_de_acciones(path_ucs)
    print("Movimientos:", camino_de_acciones)

else:
    print("No se encontró solución.")
print("Nodos explorados:", nodes_explored_ucs)

# Ejecutar Greedy
path_greedy, nodes_explored_greedy = greedy_search(maze, start_position, goal_position)
print("\n===== Búsqueda Greedy =====")
if path_greedy:
    print("Camino más corto:", path_greedy)
    camino_de_acciones = obtener_camino_de_acciones(path_greedy)
    print("Movimientos:", camino_de_acciones)
else:
    print("No se encontró solución.")
print("Nodos explorados:", nodes_explored_greedy)

print("")

# Ejecutar A*
path_a_star, nodes_explored_a_star = a_star(maze, start_position, goal_position)
print("\n===== Búsqueda A* =====")
if path_a_star:
    print("Camino más corto:", path_a_star)
    camino_de_acciones = obtener_camino_de_acciones(path_a_star)
    print("Movimientos:", camino_de_acciones)
else:
    print("No se encontró solución.")
print("Nodos explorados:", nodes_explored_a_star)


===== Búsqueda por Amplitud (BFS) =====
Movimientos: ['Abajo', 'Abajo', 'Derecha', 'Derecha', 'Abajo', 'Derecha', 'Abajo', 'Abajo', 'Derecha', 'Derecha', 'Abajo', 'Abajo']
Estados del camino: [(1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (3, 3), (4, 3), (5, 3), (5, 4), (5, 5), (6, 5), (7, 5)]
Nodos explorados: [(0, 0), (1, 0), (2, 0), (0, 0), (2, 1), (3, 0), (1, 0), (2, 2), (2, 0), (3, 1), (3, 1), (4, 0), (2, 0), (2, 1), (3, 2), (1, 2), (3, 2), (3, 0), (2, 1), (5, 0), (3, 0), (3, 3), (3, 1), (4, 2), (2, 2), (2, 2), (0, 2), (5, 1), (6, 0), (4, 0), (3, 4), (3, 2), (4, 3), (4, 3), (5, 2), (3, 2), (0, 3), (1, 2), (5, 2), (5, 0), (7, 0), (5, 0), (3, 3), (2, 4), (4, 2), (5, 3), (3, 3), (5, 3), (5, 1), (4, 2), (0, 4), (0, 2), (7, 1), (6, 0), (2, 5), (3, 4), (5, 4), (5, 2), (4, 3), (0, 5), (0, 3), (7, 2), (7, 0), (2, 4), (1, 5), (5, 5), (5, 3), (0, 4), (1, 5), (7, 3), (7, 1), (2, 5), (0, 5), (5, 4), (6, 5), (4, 5), (7, 4), (7, 2), (7, 5)]

===== Búsqueda por Profundidad (DFS) =====
Movimientos: ['

Interfaz